In [3]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go


# Load the datasets
alberta_df = pd.read_csv(r"C:\Users\Dell\Desktop\Final project\Alberta.csv")
manitoba_df = pd.read_csv(r"C:\Users\Dell\Desktop\Final project\Manitoba.csv")
ontario_df = pd.read_csv(r"C:\Users\Dell\Desktop\Final project\Ontario.csv")
quebec_df = pd.read_csv(r"C:\Users\Dell\Desktop\Final project\Quebec.csv")
vancouver_df = pd.read_csv(r"C:\Users\Dell\Desktop\Final project\Vancouver.csv")


# Add Canadian cities
alberta_df['city'] = 'Alberta'
manitoba_df['city'] = 'Manitoba'
ontario_df['city'] = 'Ontario'
quebec_df['city'] = 'Quebec'
vancouver_df['city'] = 'Vancouver'

# Combine all DataFrames
df = pd.concat([alberta_df, manitoba_df, ontario_df, quebec_df, vancouver_df])

# Create a dictionary to hold city coordinates for the map (Canada)
city_coords = {
  
    'Alberta': (53.9333, -116.5765),
    'Manitoba': (49.8951, -97.1384),
    'Ontario': (51.2538, -85.3232),
    'Quebec': (52.9399, -73.5491),
    'Vancouver': (49.2827, -123.1207)
}

# Initialize the Dash app
app = Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("Restaurant Data Analysis"),
    
    # Dropdown for selecting city
    dcc.Dropdown(
        id='city-dropdown',
        options=[{'label': city, 'value': city} for city in city_coords.keys()],
        value='Chicago',
        clearable=False
    ),
    
    # Filter fields for rating and number_of_reviews, initially hidden
    html.Div(id='filters-container', style={'display': 'none'}, children=[
        html.Label("Rating Range"),
        dcc.RangeSlider(
            id='rating-slider',
            min=0, max=5, step=0.5, value=[0, 5],
            marks={i: str(i) for i in range(6)}
        ),
        
        html.Label("Number of Reviews Range"),
        dcc.RangeSlider(
            id='review-slider',
            min=0, max=12000, step=500, value=[0, 1000],
            marks={i: f"{i}" for i in range(0, 12001, 1000)}
        ),
        
        # Dropdown to select a restaurant from the top 10 based on filters
        html.Label("Select a Restaurant"),
        dcc.Dropdown(id='restaurant-dropdown', clearable=False)
    ]),
    
    # Placeholder for restaurant details (initially empty)
    html.Div(id='graphs-container')
])

# Callback to show filters when a city is selected
@app.callback(
    Output('filters-container', 'style'),
    Input('city-dropdown', 'value')
)
def toggle_filters(selected_city):
    if selected_city:
        return {'display': 'block'}
    else:
        return {'display': 'none'}

# Callback to update the top 10 restaurants dropdown based on city and filters
@app.callback(
    Output('restaurant-dropdown', 'options'),
    Input('city-dropdown', 'value'),
    Input('rating-slider', 'value'),
    Input('review-slider', 'value')
)
def update_restaurant_options(selected_city, rating_range, review_range):
    min_rating, max_rating = rating_range
    min_reviews, max_reviews = review_range
    
    # Filter data based on city and filters
    filtered_df = df[(df['city'] == selected_city) & 
                     (df['rating'] >= min_rating) & (df['rating'] <= max_rating) &
                     (df['number_of_reviews'] >= min_reviews) & (df['number_of_reviews'] <= max_reviews)]
    
    # Get the top 10 restaurants by rating
    top_10_df = filtered_df.nlargest(10, 'rating')
    
    # Return options for the restaurant dropdown
    return [{'label': name, 'value': name} for name in top_10_df['rest_name']]

# Callback to update the restaurant details based on selected restaurant
@app.callback(
    Output('graphs-container', 'children'),
    Input('restaurant-dropdown', 'value'),
    Input('city-dropdown', 'value')
)
def update_restaurant_details(selected_restaurant, selected_city):
    if not selected_restaurant:
        return []  # No display if no restaurant is selected
    
    # Filter data to get the specific restaurant's details
    restaurant_data = df[(df['city'] == selected_city) & (df['rest_name'] == selected_restaurant)].iloc[0]
    
    # Extract specific fields
    image_url = restaurant_data['image_url']
    about_rest = restaurant_data['about_rest']
    ambience = restaurant_data['ambience']
    food = restaurant_data['food']
    service = restaurant_data['service']
    value = restaurant_data['value']
    
    # Display the restaurant's image, about section, and ratings
    return [
        html.Img(src=image_url, style={'width': '100%', 'height': 'auto'}),
        html.H3("About the Restaurant"),
        html.P(about_rest),
        html.H3("Ratings"),
        html.P(f"Ambience: {ambience}/5"),
        html.P(f"Food: {food}/5"),
        html.P(f"Service: {service}/5"),
        html.P(f"Value: {value}/5")
    ]

if __name__ == '__main__':
    app.run_server(debug=True, port=80)
